# Chapter 11

In [4]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "deep"

def save_fig(fig_id, tight_layout=True):
    path = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID, fig_id + ".png")
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format='png', dpi=300)

In [2]:
import tensorflow as tf

### 8a

In [6]:
he_init = tf.variance_scaling_initializer()

def dnn(inputs, n_hidden_layers=5, n_neurons=100, name=None,
        activation=tf.nn.elu, initializer=he_init):
    with tf.variable_scope(name, "dnn"):
        for layer in range(n_hidden_layers):
            inputs = tf.layers.dense(inputs, n_neurons, activation=activation,
                                     kernel_initializer=initializer,
                                     name="hidden%d" % (layer + 1))
        return inputs

In [7]:
n_inputs = 28 * 28 # MNIST
n_outputs = 5

reset_graph()

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

dnn_outputs = dnn(X)

logits = tf.layers.dense(dnn_outputs, n_outputs, kernel_initializer=he_init, name="logits")
Y_proba = tf.nn.softmax(logits, name="Y_proba")

### 8b

In [8]:
learning_rate = 0.01

xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
loss = tf.reduce_mean(xentropy, name="loss")

optimizer = tf.train.AdamOptimizer(learning_rate)
training_op = optimizer.minimize(loss, name="training_op")

correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [11]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)
X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]

In [12]:
X_train1 = X_train[y_train < 5]
y_train1 = y_train[y_train < 5]
X_valid1 = X_valid[y_valid < 5]
y_valid1 = y_valid[y_valid < 5]
X_test1 = X_test[y_test < 5]
y_test1 = y_test[y_test < 5]

In [13]:
n_epochs = 1000
batch_size = 20

max_checks_without_progress = 20
checks_without_progress = 0
best_loss = np.infty

with tf.Session() as sess:
    init.run()

    for epoch in range(n_epochs):
        rnd_idx = np.random.permutation(len(X_train1))
        for rnd_indices in np.array_split(rnd_idx, len(X_train1) // batch_size):
            X_batch, y_batch = X_train1[rnd_indices], y_train1[rnd_indices]
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        loss_val, acc_val = sess.run([loss, accuracy], feed_dict={X: X_valid1, y: y_valid1})
        if loss_val < best_loss:
            save_path = saver.save(sess, "./my_mnist_model_0_to_4.ckpt")
            best_loss = loss_val
            checks_without_progress = 0
        else:
            checks_without_progress += 1
            if checks_without_progress > max_checks_without_progress:
                print("Early stopping!")
                break
        print("{}\tValidation loss: {:.6f}\tBest loss: {:.6f}\tAccuracy: {:.2f}%".format(
            epoch, loss_val, best_loss, acc_val * 100))

with tf.Session() as sess:
    saver.restore(sess, "./my_mnist_model_0_to_4.ckpt")
    acc_test = accuracy.eval(feed_dict={X: X_test1, y: y_test1})
    print("Final test accuracy: {:.2f}%".format(acc_test * 100))

0	Validation loss: 0.351242	Best loss: 0.351242	Accuracy: 90.58%
1	Validation loss: 0.111183	Best loss: 0.111183	Accuracy: 97.38%
2	Validation loss: 0.209247	Best loss: 0.111183	Accuracy: 96.68%
3	Validation loss: 0.105242	Best loss: 0.105242	Accuracy: 97.89%
4	Validation loss: 0.144679	Best loss: 0.105242	Accuracy: 97.58%
5	Validation loss: 0.399149	Best loss: 0.105242	Accuracy: 94.25%
6	Validation loss: 1.397254	Best loss: 0.105242	Accuracy: 50.74%
7	Validation loss: 0.451237	Best loss: 0.105242	Accuracy: 75.06%
8	Validation loss: 0.349017	Best loss: 0.105242	Accuracy: 93.39%
9	Validation loss: 0.344995	Best loss: 0.105242	Accuracy: 94.80%
10	Validation loss: 0.176267	Best loss: 0.105242	Accuracy: 97.38%
11	Validation loss: 0.239872	Best loss: 0.105242	Accuracy: 96.99%
12	Validation loss: 1.687223	Best loss: 0.105242	Accuracy: 22.01%
13	Validation loss: 1.617635	Best loss: 0.105242	Accuracy: 22.01%
14	Validation loss: 1.662950	Best loss: 0.105242	Accuracy: 20.91%
15	Validation loss: 

### 8c

In [14]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.exceptions import NotFittedError

class DNNClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, n_hidden_layers=5, n_neurons=100, optimizer_class=tf.train.AdamOptimizer,
                 learning_rate=0.01, batch_size=20, activation=tf.nn.elu, initializer=he_init,
                 batch_norm_momentum=None, dropout_rate=None, random_state=None):
        """Initialize the DNNClassifier by simply storing all the hyperparameters."""
        self.n_hidden_layers = n_hidden_layers
        self.n_neurons = n_neurons
        self.optimizer_class = optimizer_class
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.activation = activation
        self.initializer = initializer
        self.batch_norm_momentum = batch_norm_momentum
        self.dropout_rate = dropout_rate
        self.random_state = random_state
        self._session = None

    def _dnn(self, inputs):
        """Build the hidden layers, with support for batch normalization and dropout."""
        for layer in range(self.n_hidden_layers):
            if self.dropout_rate:
                inputs = tf.layers.dropout(inputs, self.dropout_rate, training=self._training)
            inputs = tf.layers.dense(inputs, self.n_neurons,
                                     kernel_initializer=self.initializer,
                                     name="hidden%d" % (layer + 1))
            if self.batch_norm_momentum:
                inputs = tf.layers.batch_normalization(inputs, momentum=self.batch_norm_momentum,
                                                       training=self._training)
            inputs = self.activation(inputs, name="hidden%d_out" % (layer + 1))
        return inputs

    def _build_graph(self, n_inputs, n_outputs):
        """Build the same model as earlier"""
        if self.random_state is not None:
            tf.set_random_seed(self.random_state)
            np.random.seed(self.random_state)

        X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
        y = tf.placeholder(tf.int32, shape=(None), name="y")

        if self.batch_norm_momentum or self.dropout_rate:
            self._training = tf.placeholder_with_default(False, shape=(), name='training')
        else:
            self._training = None

        dnn_outputs = self._dnn(X)

        logits = tf.layers.dense(dnn_outputs, n_outputs, kernel_initializer=he_init, name="logits")
        Y_proba = tf.nn.softmax(logits, name="Y_proba")

        xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,
                                                                  logits=logits)
        loss = tf.reduce_mean(xentropy, name="loss")

        optimizer = self.optimizer_class(learning_rate=self.learning_rate)
        training_op = optimizer.minimize(loss)

        correct = tf.nn.in_top_k(logits, y, 1)
        accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

        init = tf.global_variables_initializer()
        saver = tf.train.Saver()

        # Make the important operations available easily through instance variables
        self._X, self._y = X, y
        self._Y_proba, self._loss = Y_proba, loss
        self._training_op, self._accuracy = training_op, accuracy
        self._init, self._saver = init, saver

    def close_session(self):
        if self._session:
            self._session.close()

    def _get_model_params(self):
        """Get all variable values (used for early stopping, faster than saving to disk)"""
        with self._graph.as_default():
            gvars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
        return {gvar.op.name: value for gvar, value in zip(gvars, self._session.run(gvars))}

    def _restore_model_params(self, model_params):
        """Set all variables to the given values (for early stopping, faster than loading from disk)"""
        gvar_names = list(model_params.keys())
        assign_ops = {gvar_name: self._graph.get_operation_by_name(gvar_name + "/Assign")
                      for gvar_name in gvar_names}
        init_values = {gvar_name: assign_op.inputs[1] for gvar_name, assign_op in assign_ops.items()}
        feed_dict = {init_values[gvar_name]: model_params[gvar_name] for gvar_name in gvar_names}
        self._session.run(assign_ops, feed_dict=feed_dict)

    def fit(self, X, y, n_epochs=100, X_valid=None, y_valid=None):
        """Fit the model to the training set. If X_valid and y_valid are provided, use early stopping."""
        self.close_session()

        # infer n_inputs and n_outputs from the training set.
        n_inputs = X.shape[1]
        self.classes_ = np.unique(y)
        n_outputs = len(self.classes_)
        
        # Translate the labels vector to a vector of sorted class indices, containing
        # integers from 0 to n_outputs - 1.
        # For example, if y is equal to [8, 8, 9, 5, 7, 6, 6, 6], then the sorted class
        # labels (self.classes_) will be equal to [5, 6, 7, 8, 9], and the labels vector
        # will be translated to [3, 3, 4, 0, 2, 1, 1, 1]
        self.class_to_index_ = {label: index
                                for index, label in enumerate(self.classes_)}
        y = np.array([self.class_to_index_[label]
                      for label in y], dtype=np.int32)
        
        self._graph = tf.Graph()
        with self._graph.as_default():
            self._build_graph(n_inputs, n_outputs)
            # extra ops for batch normalization
            extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)

        # needed in case of early stopping
        max_checks_without_progress = 20
        checks_without_progress = 0
        best_loss = np.infty
        best_params = None
        
        # Now train the model!
        self._session = tf.Session(graph=self._graph)
        with self._session.as_default() as sess:
            self._init.run()
            for epoch in range(n_epochs):
                rnd_idx = np.random.permutation(len(X))
                for rnd_indices in np.array_split(rnd_idx, len(X) // self.batch_size):
                    X_batch, y_batch = X[rnd_indices], y[rnd_indices]
                    feed_dict = {self._X: X_batch, self._y: y_batch}
                    if self._training is not None:
                        feed_dict[self._training] = True
                    sess.run(self._training_op, feed_dict=feed_dict)
                    if extra_update_ops:
                        sess.run(extra_update_ops, feed_dict=feed_dict)
                if X_valid is not None and y_valid is not None:
                    loss_val, acc_val = sess.run([self._loss, self._accuracy],
                                                 feed_dict={self._X: X_valid,
                                                            self._y: y_valid})
                    if loss_val < best_loss:
                        best_params = self._get_model_params()
                        best_loss = loss_val
                        checks_without_progress = 0
                    else:
                        checks_without_progress += 1
                    print("{}\tValidation loss: {:.6f}\tBest loss: {:.6f}\tAccuracy: {:.2f}%".format(
                        epoch, loss_val, best_loss, acc_val * 100))
                    if checks_without_progress > max_checks_without_progress:
                        print("Early stopping!")
                        break
                else:
                    loss_train, acc_train = sess.run([self._loss, self._accuracy],
                                                     feed_dict={self._X: X_batch,
                                                                self._y: y_batch})
                    print("{}\tLast training batch loss: {:.6f}\tAccuracy: {:.2f}%".format(
                        epoch, loss_train, acc_train * 100))
            # If we used early stopping then rollback to the best model found
            if best_params:
                self._restore_model_params(best_params)
            return self

    def predict_proba(self, X):
        if not self._session:
            raise NotFittedError("This %s instance is not fitted yet" % self.__class__.__name__)
        with self._session.as_default() as sess:
            return self._Y_proba.eval(feed_dict={self._X: X})

    def predict(self, X):
        class_indices = np.argmax(self.predict_proba(X), axis=1)
        return np.array([[self.classes_[class_index]]
                         for class_index in class_indices], np.int32)

    def save(self, path):
        self._saver.save(self._session, path)

In [15]:
dnn_clf = DNNClassifier(random_state=42)
dnn_clf.fit(X_train1, y_train1, n_epochs=1000, X_valid=X_valid1, y_valid=y_valid1)

0	Validation loss: 0.351242	Best loss: 0.351242	Accuracy: 90.58%
1	Validation loss: 0.111183	Best loss: 0.111183	Accuracy: 97.38%
2	Validation loss: 0.209247	Best loss: 0.111183	Accuracy: 96.68%
3	Validation loss: 0.105242	Best loss: 0.105242	Accuracy: 97.89%
4	Validation loss: 0.144679	Best loss: 0.105242	Accuracy: 97.58%
5	Validation loss: 0.399149	Best loss: 0.105242	Accuracy: 94.25%
6	Validation loss: 1.397254	Best loss: 0.105242	Accuracy: 50.74%
7	Validation loss: 0.451237	Best loss: 0.105242	Accuracy: 75.06%
8	Validation loss: 0.349017	Best loss: 0.105242	Accuracy: 93.39%
9	Validation loss: 0.344995	Best loss: 0.105242	Accuracy: 94.80%
10	Validation loss: 0.176267	Best loss: 0.105242	Accuracy: 97.38%
11	Validation loss: 0.239872	Best loss: 0.105242	Accuracy: 96.99%
12	Validation loss: 1.687223	Best loss: 0.105242	Accuracy: 22.01%
13	Validation loss: 1.617635	Best loss: 0.105242	Accuracy: 22.01%
14	Validation loss: 1.662950	Best loss: 0.105242	Accuracy: 20.91%
15	Validation loss: 

DNNClassifier(activation=<function elu at 0x0000027FD9FA1F28>,
       batch_norm_momentum=None, batch_size=20, dropout_rate=None,
       initializer=<tensorflow.python.ops.init_ops.VarianceScaling object at 0x0000027FE91CA0F0>,
       learning_rate=0.01, n_hidden_layers=5, n_neurons=100,
       optimizer_class=<class 'tensorflow.python.training.adam.AdamOptimizer'>,
       random_state=42)

In [17]:
from sklearn.model_selection import RandomizedSearchCV

def leaky_relu(alpha=0.01):
    def parametrized_leaky_relu(z, name=None):
        return tf.maximum(alpha * z, z, name=name)
    return parametrized_leaky_relu

param_distribs = {
    "n_neurons": [10, 30, 50, 70, 90, 100, 120, 140, 160],
    "batch_size": [10, 50, 100, 500],
    "learning_rate": [0.01, 0.02, 0.05, 0.1],
    "activation": [tf.nn.relu, tf.nn.elu, leaky_relu(alpha=0.01), leaky_relu(alpha=0.1)],
    # you could also try exploring different numbers of hidden layers, different optimizers, etc.
    #"n_hidden_layers": [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    #"optimizer_class": [tf.train.AdamOptimizer, partial(tf.train.MomentumOptimizer, momentum=0.95)],
}

rnd_search = RandomizedSearchCV(DNNClassifier(random_state=42), param_distribs, n_iter=50,
                                cv=3, random_state=42, verbose=2)
rnd_search.fit(X_train1, y_train1, X_valid=X_valid1, y_valid=y_valid1, n_epochs=1000)

# If you have Scikit-Learn 0.18 or earlier, you should upgrade, or use the fit_params argument:
# fit_params = dict(X_valid=X_valid1, y_valid=y_valid1, n_epochs=1000)
# rnd_search = RandomizedSearchCV(DNNClassifier(random_state=42), param_distribs, n_iter=50,
#                                 fit_params=fit_params, random_state=42, verbose=2)
# rnd_search.fit(X_train1, y_train1)

Fitting 3 folds for each of 50 candidates, totalling 150 fits
[CV] n_neurons=10, learning_rate=0.05, batch_size=100, activation=<function elu at 0x0000027FD9FA1F28> 
0	Validation loss: 0.148073	Best loss: 0.148073	Accuracy: 95.82%
1	Validation loss: 0.172988	Best loss: 0.148073	Accuracy: 95.11%
2	Validation loss: 0.106110	Best loss: 0.106110	Accuracy: 97.11%
3	Validation loss: 0.141213	Best loss: 0.106110	Accuracy: 96.76%
4	Validation loss: 0.120116	Best loss: 0.106110	Accuracy: 96.60%
5	Validation loss: 0.150639	Best loss: 0.106110	Accuracy: 96.68%
6	Validation loss: 0.158447	Best loss: 0.106110	Accuracy: 96.36%
7	Validation loss: 0.110517	Best loss: 0.106110	Accuracy: 96.83%
8	Validation loss: 0.098133	Best loss: 0.098133	Accuracy: 97.42%
9	Validation loss: 0.149589	Best loss: 0.098133	Accuracy: 97.03%
10	Validation loss: 0.145655	Best loss: 0.098133	Accuracy: 96.91%
11	Validation loss: 1.650010	Best loss: 0.098133	Accuracy: 25.18%
12	Validation loss: 1.037138	Best loss: 0.098133	Acc

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    7.1s remaining:    0.0s


0	Validation loss: 0.153572	Best loss: 0.153572	Accuracy: 95.93%
1	Validation loss: 0.136263	Best loss: 0.136263	Accuracy: 96.79%
2	Validation loss: 0.162667	Best loss: 0.136263	Accuracy: 96.13%
3	Validation loss: 0.114553	Best loss: 0.114553	Accuracy: 97.07%
4	Validation loss: 0.117696	Best loss: 0.114553	Accuracy: 96.95%
5	Validation loss: 0.116428	Best loss: 0.114553	Accuracy: 97.30%
6	Validation loss: 0.105384	Best loss: 0.105384	Accuracy: 97.11%
7	Validation loss: 0.276382	Best loss: 0.105384	Accuracy: 90.81%
8	Validation loss: 1.614943	Best loss: 0.105384	Accuracy: 22.01%
9	Validation loss: 1.615130	Best loss: 0.105384	Accuracy: 22.01%
10	Validation loss: 1.615981	Best loss: 0.105384	Accuracy: 22.01%
11	Validation loss: 1.609805	Best loss: 0.105384	Accuracy: 22.01%
12	Validation loss: 1.615424	Best loss: 0.105384	Accuracy: 19.27%
13	Validation loss: 1.622959	Best loss: 0.105384	Accuracy: 19.08%
14	Validation loss: 1.619082	Best loss: 0.105384	Accuracy: 22.01%
15	Validation loss: 

25	Validation loss: 0.077444	Best loss: 0.055630	Accuracy: 98.48%
26	Validation loss: 0.074994	Best loss: 0.055630	Accuracy: 98.24%
27	Validation loss: 0.074597	Best loss: 0.055630	Accuracy: 98.59%
28	Validation loss: 0.079588	Best loss: 0.055630	Accuracy: 98.40%
29	Validation loss: 0.079672	Best loss: 0.055630	Accuracy: 98.55%
Early stopping!
[CV]  n_neurons=30, learning_rate=0.02, batch_size=500, activation=<function relu at 0x0000027FD9F922F0>, total=   6.5s
[CV] n_neurons=30, learning_rate=0.02, batch_size=500, activation=<function relu at 0x0000027FD9F922F0> 
0	Validation loss: 0.101571	Best loss: 0.101571	Accuracy: 96.79%
1	Validation loss: 0.077616	Best loss: 0.077616	Accuracy: 97.97%
2	Validation loss: 0.059689	Best loss: 0.059689	Accuracy: 98.28%
3	Validation loss: 0.057170	Best loss: 0.057170	Accuracy: 98.48%
4	Validation loss: 0.051170	Best loss: 0.051170	Accuracy: 98.48%
5	Validation loss: 0.074578	Best loss: 0.051170	Accuracy: 97.81%
6	Validation loss: 0.061382	Best loss: 

17	Validation loss: 10.315502	Best loss: 0.203149	Accuracy: 86.90%
18	Validation loss: 220.562866	Best loss: 0.203149	Accuracy: 67.79%
19	Validation loss: 89.006546	Best loss: 0.203149	Accuracy: 86.94%
20	Validation loss: 75.730438	Best loss: 0.203149	Accuracy: 87.76%
21	Validation loss: 58.739140	Best loss: 0.203149	Accuracy: 90.30%
22	Validation loss: 538.525757	Best loss: 0.203149	Accuracy: 58.48%
23	Validation loss: 74.990616	Best loss: 0.203149	Accuracy: 88.12%
24	Validation loss: 49.641582	Best loss: 0.203149	Accuracy: 90.85%
25	Validation loss: 40.597614	Best loss: 0.203149	Accuracy: 89.52%
26	Validation loss: 30.105947	Best loss: 0.203149	Accuracy: 92.69%
27	Validation loss: 5881.931641	Best loss: 0.203149	Accuracy: 36.98%
28	Validation loss: 782.946960	Best loss: 0.203149	Accuracy: 46.13%
29	Validation loss: 97.522377	Best loss: 0.203149	Accuracy: 67.47%
Early stopping!
[CV]  n_neurons=90, learning_rate=0.05, batch_size=50, activation=<function leaky_relu.<locals>.parametrized

...

31	Validation loss: 0.523954	Best loss: 0.523954	Accuracy: 97.22%
32	Validation loss: 0.631730	Best loss: 0.523954	Accuracy: 97.73%
33	Validation loss: 0.655092	Best loss: 0.523954	Accuracy: 95.93%
34	Validation loss: 1.419322	Best loss: 0.523954	Accuracy: 94.37%
35	Validation loss: 0.909884	Best loss: 0.523954	Accuracy: 97.11%
36	Validation loss: 1.087830	Best loss: 0.523954	Accuracy: 97.07%
37	Validation loss: 0.711218	Best loss: 0.523954	Accuracy: 96.99%
38	Validation loss: 0.942398	Best loss: 0.523954	Accuracy: 97.34%
39	Validation loss: 1.235979	Best loss: 0.523954	Accuracy: 96.68%
40	Validation loss: 0.912256	Best loss: 0.523954	Accuracy: 97.46%
41	Validation loss: 0.864100	Best loss: 0.523954	Accuracy: 96.79%
42	Validation loss: 0.796213	Best loss: 0.523954	Accuracy: 97.54%
43	Validation loss: 0.783654	Best loss: 0.523954	Accuracy: 97.81%
44	Validation loss: 0.761767	Best loss: 0.523954	Accuracy: 96.99%
45	Validation loss: 0.869086	Best loss: 0.523954	Accuracy: 97.07%
46	Validat

38	Validation loss: 46380.476562	Best loss: 0.721018	Accuracy: 70.37%
39	Validation loss: 23877.541016	Best loss: 0.721018	Accuracy: 82.72%
40	Validation loss: 18223.763672	Best loss: 0.721018	Accuracy: 83.15%
41	Validation loss: 21077.337891	Best loss: 0.721018	Accuracy: 79.20%
42	Validation loss: 14445.995117	Best loss: 0.721018	Accuracy: 87.26%
43	Validation loss: 11619.138672	Best loss: 0.721018	Accuracy: 87.18%
Early stopping!
[CV]  n_neurons=90, learning_rate=0.1, batch_size=500, activation=<function leaky_relu.<locals>.parametrized_leaky_relu at 0x0000027FEC22EE18>, total=  16.8s


[Parallel(n_jobs=1)]: Done 150 out of 150 | elapsed: 67.9min finished


0	Validation loss: 0.069587	Best loss: 0.069587	Accuracy: 98.12%
1	Validation loss: 0.045462	Best loss: 0.045462	Accuracy: 98.48%
2	Validation loss: 0.046386	Best loss: 0.045462	Accuracy: 98.40%
3	Validation loss: 0.045583	Best loss: 0.045462	Accuracy: 98.36%
4	Validation loss: 0.066225	Best loss: 0.045462	Accuracy: 97.93%
5	Validation loss: 0.040087	Best loss: 0.040087	Accuracy: 98.75%
6	Validation loss: 0.034379	Best loss: 0.034379	Accuracy: 98.98%
7	Validation loss: 0.034873	Best loss: 0.034379	Accuracy: 98.83%
8	Validation loss: 0.030992	Best loss: 0.030992	Accuracy: 99.18%
9	Validation loss: 0.041969	Best loss: 0.030992	Accuracy: 98.87%
10	Validation loss: 0.032636	Best loss: 0.030992	Accuracy: 98.94%
11	Validation loss: 0.064768	Best loss: 0.030992	Accuracy: 98.44%
12	Validation loss: 0.037094	Best loss: 0.030992	Accuracy: 98.91%
13	Validation loss: 0.045894	Best loss: 0.030992	Accuracy: 98.91%
14	Validation loss: 0.064599	Best loss: 0.030992	Accuracy: 98.59%
15	Validation loss: 

RandomizedSearchCV(cv=3, error_score='raise',
          estimator=DNNClassifier(activation=<function elu at 0x0000027FD9FA1F28>,
       batch_norm_momentum=None, batch_size=20, dropout_rate=None,
       initializer=<tensorflow.python.ops.init_ops.VarianceScaling object at 0x0000027FE91CA0F0>,
       learning_rate=0.01, n_hidden_layers=5, n_neurons=100,
       optimizer_class=<class 'tensorflow.python.training.adam.AdamOptimizer'>,
       random_state=42),
          fit_params=None, iid=True, n_iter=50, n_jobs=1,
          param_distributions={'n_neurons': [10, 30, 50, 70, 90, 100, 120, 140, 160], 'batch_size': [10, 50, 100, 500], 'learning_rate': [0.01, 0.02, 0.05, 0.1], 'activation': [<function relu at 0x0000027FD9F922F0>, <function elu at 0x0000027FD9FA1F28>, <function leaky_relu.<locals>.parametrized_leaky_relu at 0x0000027FE924EB70>, <function leaky_relu.<locals>.parametrized_leaky_relu at 0x0000027FEC22EE18>]},
          pre_dispatch='2*n_jobs', random_state=42, refit=True,
      